In [1]:
import pandas as pd
import requests
import json
import time
import numpy as np

####
# Python request from Postman API

payload={}
headers = {
  'authority': 'storefrontgateway.saveonfoods.com',
  'accept': 'application/json, text/plain, */*',
  'accept-language': 'en-US,en;q=0.9,fr;q=0.8,es;q=0.7',
  'cookie': '__cf_bm=wjMzBNi6PxW3p8HbA._PJChcnv6T_REpq8c7frGi264-1663479712-0-AauPv/dYMV18XsPuadNC/lRr40/G3RBnbBhsDuSAlqhK2IbVKNx/dSRfwbSqDEoD5ZRtHM4NBX1zo/mVw8CyaM4tUOdekuruZX4OegOcJolvHeLGKElv0RySMZSlDMWIA1Df/mFwWHnIMUoZUSXgKYlJKNtPa9YUlBtxKBh4TuOGlJFFE6o0GBdveDqsZw+ZWQ==; _gcl_au=1.1.1520443345.1663479713; _ga=GA1.2.1922336475.1663479713; _gid=GA1.2.764078445.1663479713; _dc_gtm_UA-3016345-8=1; _gaexp=GAX1.2.2TXrwbPLRTqBNA5IC8es5w.19316.0; _fbp=fb.1.1663479713691.620429690; AMCVS_9C423E945A72D9EE0A495EF8%40AdobeOrg=1; _pin_unauth=dWlkPU5qQm1NMlJoTmpRdE56ZzRPUzAwWkRSbExUaG1NVGd0WkRoaFpESXpZVGszTWpFMg; AMCV_9C423E945A72D9EE0A495EF8%40AdobeOrg=1585540135%7CMCMID%7C26107693206724934624409335891349591500%7CMCAAMLH-1664084513%7C9%7CMCAAMB-1664084513%7C6G1ynYcLPuiQxYZrsz_pkqfLG9yMXBpb2zX5dvJdYQJzPXImdj0y%7CMCCIDH%7C-2127053844%7CMCOPTOUT-1663486913s%7CNONE%7CvVersion%7C4.4.0; _ga_ZM4WBLPQ9B=GS1.1.1663479713.1.1.1663479725.0.0.0',
  'customerid': 'unknown',
  'dnt': '1',
  'origin': 'https://www.saveonfoods.com',
  'referer': 'https://www.saveonfoods.com/',
  'sec-ch-ua': '"Google Chrome";v="105", "Not)A;Brand";v="8", "Chromium";v="105"',
  'sec-ch-ua-mobile': '?0',
  'sec-ch-ua-platform': '"Windows"',
  'sec-fetch-dest': 'empty',
  'sec-fetch-mode': 'cors',
  'sec-fetch-site': 'same-site',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
  'x-correlation-id': '7984c467-d650-40de-a536-fdb2fba88de0',
  'x-shopping-mode': '11111111-1111-1111-1111-111111111111',
  'x-site-host': 'https://www.saveonfoods.com'
}

###

# define function to scrape

def save_on_scrape_api(name, skip=30):
    '''
    input the search term into Save-On-Foods API and extract data

    name: search term (e.g. meat, plant-based) in string type.
    skip: integer in the url API. Used for pagination or scrolling through the data results. default = 30
    Returns a pandas dataframe
    '''
    
    df = pd.DataFrame([]) # create blank dataframe to store data
    
    while True:
        print('Scrapping...')
        url = f"https://storefrontgateway.saveonfoods.com/api/stores/1982/search?q={name}&take=30&skip={skip}&page=1"
        r = requests.get(url, headers=headers, data=payload)
        data = json.loads(r.text)
        data_df = pd.json_normalize(data['items'], 'defaultCategory', ['productId', 'sku', 'name', 'description',
                                                    'brand', 'priceNumeric' , 'wasPriceNumeric','pricePerUnit',
                                                    ['unitOfMeasure', 'type']], errors='ignore')
        df = df.append(data_df)

        if len(data['items']) == 0:
               break

        skip += 30 # pagination increment. Allows to "scroll" through the data
        
    
        
    print('Done scrapping')
    print('Len database: ' + str(len(df)))
    
    # clean and modify dataframe
    df.rename(columns={'categoryBreadcrumb': 'categories', 'priceNumeric': 'price', 
                       'wasPriceNumeric': 'prev_price', 
                       'pricePerUnit': 'price_unit', 
                       'unitOfMeasure.type': 'unit'}, inplace=True)

    split = df.price_unit.str.split('/', expand=True)
    df['price_unit_0'] = split[0]
    df['price_unit_1'] = split[1]

    df['price_unit_0'] = df.price_unit_0.str.replace('$', '', regex=True)
    df['price_unit_0'] = df.price_unit_0.str.replace('each', '', regex=True)
    df['price_unit_1'] = df.price_unit_1.str.replace('g', '', regex=True)
    
    df.fillna(np.nan, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

In [2]:
df = save_on_scrape_api('meat')

Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Scrapping...
Done scrapping
Len database: 1519


In [3]:
# save dataframe to csv file
df.to_csv('save-on-foods-meat-products.csv')

In [4]:
df.head()

,categoryId,retailerId,category,categories,productId,sku,name,description,brand,price,prev_price,price_unit,unit,price_unit_0,price_unit_1
0,4c171bbd-ab9d-497f-ab48-9ed6eb8229db,30796,Beef Steak Cuts,Grocery/Meat & Seafood/Beef & Veal/Beef Steak ...,00236234000008,00236234000008,Western Canadian - Sirloin Tip Marinating Stea...,Cut From AAA Western Canadian Beef. Aged Min. ...,Western Canadian,18.500311,NaN,$2.20/100g,gram,2.20,100
1,9266091f-9f48-48d6-9f36-b1b69e542b0a,30808,Ground Pork,Grocery/Meat & Seafood/Pork & Ham/Ground Pork,00077782001006,00077782001006,Johnsonville - Italiano Mild Italian Sausage Meat,Gluten Free. Ground Meat Perfect Blend of Herb...,Johnsonville,6.990000,NaN,$1.86/100g,gram,1.86,100
2,71479d85-44cf-45d5-828a-cfa548d82742,30819,Hot Dogs & Franks,Grocery/Meat & Seafood/Hot Dogs & Sausages/Hot...,00063100183943,00063100183943,Schneiders - Juicy Jumbos All Beef Wieners,Schneiders Juicy Jumbos All Beef Wieners are a...,Schneiders,8.990000,NaN,$2.40/100g,gram,2.40,100
3,4c171bbd-ab9d-497f-ab48-9ed6eb8229db,30796,Beef Steak Cuts,Grocery/Meat & Seafood/Beef & Veal/Beef Steak ...,00294738000009,00294738000009,Western Canadian - Flank Steak,AAA Beef. Grain Fed. Average Weight of Each Pa...,Western Canadian,13.709093,NaN,$3.75/100g,gram,3.75,100
4,4c171bbd-ab9d-497f-ab48-9ed6eb8229db,30796,Beef Steak Cuts,Grocery/Meat & Seafood/Beef & Veal/Beef Steak ...,00210410000006,00210410000006,"Western Canadian - Striploin Grilling Steak, F...",AAA Beef. Grain Fed. Average Weight of Each Pa...,Western Canadian,27.765017,NaN,$4.63/100g,gram,4.63,100


## Create Database

In [5]:
# save dataframe into database
from sqlalchemy import create_engine
from sqlalchemy.types import Text, Float, Integer

# open connection/create database file
engine = create_engine('sqlite:///plant-based-save-on-foods.db', echo=False)

# safe dataframe into a table in the database
df.to_sql('products', con=engine, if_exists='replace', index=False, dtype={'categoryId': Text(),
                                                                         'retailerId': Integer(),
                                                                         'category': Text(),
                                                                         'categories': Text(),
                                                                         'productId': Integer(),
                                                                         'sku': Integer(),
                                                                         'name': Text(),
                                                                         'description': Text(),
                                                                         'brand': Text(),
                                                                         'price': Float(),
                                                                         'prev_price': Float(),
                                                                          'price_unit': Text(),
                                                                          'unit': Text(),
                                                                         'price_unit_0': Float(),
                                                                         'price_unit_1': Integer()})

## Query the Database

In [6]:
# Load the sql module to iPython
%load_ext sql
%sql sqlite:///plant-based-save-on-foods.db

In [ ]:
%%sql

SELECT *
FROM products
WHERE retailerId = '30796';